In [ ]:
### Find coords box, barrier, and mirror - with position name

import cv2
import os
import pandas as pd
from os.path import exists

def clear_filepath(directory):
    for dirpath,_, filenames in os.walk(directory):
        for files in filenames:
            if not files.startswith("."):
                yield os.path.abspath(os.path.join(dirpath,files))

image_list_path = list(clear_filepath("images/boundary_four"))
print(image_list_path)

if os.path.exists("Boundary_coords.csv"):
    done_df = pd.read_csv("Boundary_coords.csv")
    done_pos = pd.unique(done_df.Position)
else:
    done_pos = []

df = pd.DataFrame(columns=["Name", "Position", "Point", "Coordinates"])

def slope_calc (pt1,pt2):
    x1,y1 = pt1
    x2,y2 = pt2
    if (x2-x1) == 0:
        b1 = y1
        b2 = y2
    else:
        m = (y2-y1)/(x2-x1)
        m2 = -1/m
        b1 = -(m2*x1-y1)
        b2 = -(m2*x2-y2)
    return(b1,b2)

def line_intersection(line1, line2):
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])
    
    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]
    
    div = det(xdiff, ydiff)
    if div == 0:
        raise Exception("lines do not intersect")
    
    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return x, y

def pts_mirror(coordinateStore1,lineb):
    pt1 = coordinateStore1.points[-1]
    pt2 = coordinateStore1.points[-2]
    b1,b2 = slope_calc(pt1,pt2)
    pt3 = (0,b1)
    pt4 = (0,b2)
    line1 = (pt1, pt3)
    line2 = (pt2, pt4)
    m_pt3 = line_intersection(line1, lineb)
    m_pt4 = line_intersection(line2, lineb)
    m_pt3 = (int(m_pt3[0]), int(m_pt3[1]))
    m_pt4 = (int(m_pt4[0]), int(m_pt4[1]))
    return m_pt3, m_pt4
    
class CoordinateStore:
    def __init__(self):
        self.points = []
        self.clicks = 0

    def select_point(self,event,x,y,flags,param):
            if event == cv2.EVENT_LBUTTONDOWN:
                if self.clicks <16 :
                    cv2.circle(current_image,(x,y),3,(0,255,0),-1)
                elif 16<= self.clicks <24 :
                    cv2.circle(current_image,(x,y),3,(0,0,255),-1)
                else:
                    cv2.circle(current_image,(x,y),3,(255,0,0),-1)
                self.points.append((x,y))
                self.clicks += 1
                
coordinateStore1 = CoordinateStore()

cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.setMouseCallback('image',coordinateStore1.select_point)

for image in image_list_path:
    image_name = os.path.basename(image)
    print(image_name)
    l = [idx for idx, item in enumerate(image_name.lower()) if "_" in item]
    start_pos = l[0] + 1
    end_pos = l[1]
    pos_name = image_name[start_pos:end_pos]
    if pos_name in done_pos:
        print(pos_name, "already done")
        continue
    print(pos_name)
    current_image = cv2.imread(image)
    b_tl_done = False
    b_tr_done = False
    b_bl_done = False
    b_br_done = False
    tl_done = False
    tr_done = False
    bl_done = False
    br_done = False
    while(1):
        cv2.imshow('image',current_image)
        k = cv2.waitKey(20) & 0xFF
        if coordinateStore1.clicks < 4:
            cv2.rectangle(current_image, (1510,70), (1900,0), (0,0,0), -1)
            cv2.putText(current_image, "Box top left", (1520, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 255, 0), 3)
        elif 4 <= coordinateStore1.clicks < 8:
            cv2.rectangle(current_image, (1510,70), (1900,0), (0,0,0), -1)
            cv2.putText(current_image, "Box top right", (1520, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 255, 0), 3)
        elif 8 <= coordinateStore1.clicks < 12:
            cv2.rectangle(current_image, (1510,70), (1900,0), (0,0,0), -1)
            cv2.putText(current_image, "Box bottom left", (1520, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 255, 0), 3)
        elif 12 <= coordinateStore1.clicks < 16:
            cv2.rectangle(current_image, (1510,70), (1900,0), (0,0,0), -1)
            cv2.putText(current_image, "Box bottom right", (1520, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 255, 0), 3)
        elif 16 <= coordinateStore1.clicks < 18:
            cv2.rectangle(current_image, (1510,70), (1900,0), (0,0,0), -1)
            cv2.putText(current_image, "Barrier top left", (1520, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0,0,255), 3)
        elif 18 <= coordinateStore1.clicks < 20:
            if coordinateStore1.clicks == 18:
                if b_tl_done == False:
                    b_pt1 = coordinateStore1.points[-1]
                    b_pt2 = coordinateStore1.points[-2]
                    b_tl = (b_pt1, b_pt2)
                    b_tl_done = True
            cv2.rectangle(current_image, (1510,70), (1900,0), (0,0,0), -1)
            cv2.putText(current_image, "Barrier top right", (1520, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0,0,255), 3)
        elif 20 <= coordinateStore1.clicks < 22:
            if coordinateStore1.clicks == 20:
                if b_tr_done == False:
                    b_pt1 = coordinateStore1.points[-1]
                    b_pt2 = coordinateStore1.points[-2]
                    b_tr = (b_pt1, b_pt2)
                    b_tr_done = True
            cv2.rectangle(current_image, (1510,70), (1900,0), (0,0,0), -1)
            cv2.putText(current_image, "Barrier bottom left", (1520, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0,0,255), 3)
        elif 22 <= coordinateStore1.clicks < 24:
            if coordinateStore1.clicks == 22:
                if b_bl_done == False:
                    b_pt1 = coordinateStore1.points[-1]
                    b_pt2 = coordinateStore1.points[-2]
                    b_bl = (b_pt1, b_pt2)
                    b_bl_done = True
            cv2.rectangle(current_image, (1510,70), (1900,0), (0,0,0), -1)
            cv2.putText(current_image, "Barrier bottom right", (1520, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0,0,255), 3)
        elif 24 <= coordinateStore1.clicks < 26:
            if coordinateStore1.clicks == 24:
                if b_br_done == False:
                    b_pt1 = coordinateStore1.points[-1]
                    b_pt2 = coordinateStore1.points[-2]
                    b_br = (b_pt1, b_pt2)
                    b_br_done = True
            cv2.rectangle(current_image, (1510,70), (1900,0), (0,0,0), -1)
            cv2.putText(current_image, "Mirror top left", (1520, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (255,0,0), 3)
        elif 26 <= coordinateStore1.clicks < 28:
            if coordinateStore1.clicks == 26:
                if tl_done == False:
                    m_pt3, m_pt4 = pts_mirror(coordinateStore1, b_tl)
                    coordinateStore1.points.append(m_pt3)
                    cv2.circle(current_image,m_pt3,3,(255,0,0),-1)
                    coordinateStore1.points.append(m_pt4)
                    cv2.circle(current_image,m_pt4,3,(255,0,0),-1)
                    tl_done = True
            cv2.rectangle(current_image, (1510,70), (1900,0), (0,0,0), -1)
            cv2.putText(current_image, "Mirror top right", (1520, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (255,0,0), 3)
        elif 28 <= coordinateStore1.clicks < 30:
            if coordinateStore1.clicks == 28:
                if tr_done == False:
                    m_pt3, m_pt4 = pts_mirror(coordinateStore1, b_tr)
                    coordinateStore1.points.append(m_pt3)
                    cv2.circle(current_image,m_pt3,3,(255,0,0),-1)
                    coordinateStore1.points.append(m_pt4)
                    cv2.circle(current_image,m_pt4,3,(255,0,0),-1)
                    tr_done = True
            cv2.rectangle(current_image, (1510,70), (1900,0), (0,0,0), -1)
            cv2.putText(current_image, "Mirror bottom left", (1520, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (255,0,0), 3)
        elif 30 <= coordinateStore1.clicks < 32:
            if coordinateStore1.clicks == 30:
                if bl_done == False:
                    m_pt3, m_pt4 = pts_mirror(coordinateStore1, b_bl)
                    coordinateStore1.points.append(m_pt3)
                    cv2.circle(current_image,m_pt3,3,(255,0,0),-1)
                    coordinateStore1.points.append(m_pt4)
                    cv2.circle(current_image,m_pt4,3,(255,0,0),-1)
                    bl_done = True
            cv2.rectangle(current_image, (1510,70), (1900,0), (0,0,0), -1)
            cv2.putText(current_image, "Mirror bottom right", (1520, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (255,0,0), 3)
        elif coordinateStore1.clicks == 32:
            if br_done == False:
                m_pt3, m_pt4 = pts_mirror(coordinateStore1, b_br)
                coordinateStore1.points.append(m_pt3)
                cv2.circle(current_image,m_pt3,3,(255,0,0),-1)
                coordinateStore1.points.append(m_pt4)
                cv2.circle(current_image,m_pt4,3,(255,0,0),-1)
                br_done = True
        
        if k == 32:
            count = 1
            if len(coordinateStore1.points) >= 40:
                for i in coordinateStore1.points:
                    if count > 40:
                        break
                    if count <= 16:
                        temp_df = pd.DataFrame({"Name": [image_name], "Position": [pos_name], "Point": ["box"], "Coordinates": [i]})
                    if 16 < count <= 24:
                        temp_df = pd.DataFrame({"Name": [image_name], "Position": [pos_name], "Point": ["barrier"], "Coordinates": [i]})
                    if 24 < count <= 40:
                        temp_df = pd.DataFrame({"Name": [image_name], "Position": [pos_name], "Point": ["mirror"], "Coordinates": [i]})
                    df = pd.concat([df,temp_df], ignore_index = True)
                    count += 1
            else:
                print("Not enough points selected")
            coordinateStore1.__init__()
            break
        if k == 27:
            break
            
cv2.destroyWindow('image')
cv2.destroyAllWindows()

print(df)
if os.path.exists("Boundary_coords.csv"):
    df.to_csv("Boundary_coords.csv", index=False, header=False, mode="a")
else:
    df.to_csv("Boundary_coords.csv", index=False)

print("Done")

In [ ]:
### Find lids coords - with position name

import cv2
import os
import pandas as pd

def clear_filepath(directory):
    for dirpath,_, filenames in os.walk(directory):
        for files in filenames:
            if not files.startswith("."):
                yield os.path.abspath(os.path.join(dirpath,files))

                
image_list_path = list(clear_filepath("images/with_lids"))

if os.path.exists("OF_mirror/Lid_coords.csv"):
    image_list_path2 = []
    done_df = pd.read_csv("OF_mirror/Lid_coords.csv")
    done_imgs = pd.unique(done_df.Name)
    for path in image_list_path:
        img = os.path.basename(path)
        if img not in done_imgs:
            image_list_path2.append(path)
else:
    image_list_path2 = image_list_path
    
print(image_list_path2)
print("To go through:", len(image_list_path2))

df = pd.DataFrame(columns=["Name", "Position", "Point", "Coordinates"])

class CoordinateStore:
    def __init__(self):
        self.points = []
        self.clicks = 0

    def select_point(self,event,x,y,flags,param):
            if event == cv2.EVENT_LBUTTONDOWN:
                if self.clicks == 0:
                    self.points = []
                if (self.clicks % 2) == 0:
                    cv2.circle(current_image,(x,y),3,(255,128,0),-1)
                else:
                    cv2.circle(current_image,(x,y),3,(0,0,255),-1)
                self.points.append((x,y))
                self.clicks += 1
                if self.clicks == 8:
                    self.clicks = 0
        
    def skip_point(self):
        self.points.append(("NA","NA"))
        self.clicks += 1
        if self.clicks == 8:
            self.clicks = 0
                
coordinateStore1 = CoordinateStore()

def write_boxes(image, width, box, col):
    cv2.rectangle(image, (0,70), (300,0), (0,0,0), -1)
    cv2.rectangle(image, ((width-200),70), (width,0), (0,0,0), -1)
    cv2.putText(image, box, (10,50), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (255,255,255), 3)
    if col == "Red":
        BGR = (0,0,255)
    if col == "Blue":
        BGR = (255,0,0)
    cv2.putText(image, col, ((width-190),50), cv2.FONT_HERSHEY_SIMPLEX, 1.4, BGR, 3)

cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.setMouseCallback('image',coordinateStore1.select_point)

for image in image_list_path2:
    image_name = os.path.basename(image)
    print(image_name)
    l = [idx for idx, item in enumerate(image_name.lower()) if "_" in item]
    start_pos = l[0] + 1
    end_pos = l[1]
    pos_name = image_name[start_pos:end_pos]
    print(pos_name)
    current_image = cv2.imread(image)
    width, height = current_image.shape[1], current_image.shape[0]
    Stop = False
    while(1):
        cv2.imshow('image',current_image)
        cv2.rectangle(current_image, (470,height), (width,height-70), (0,0,0), -1)
        cv2.putText(current_image, "Esc = stop code, n = next video without saving, m = missing coordinate, Space = save", 
                   (480,(height-20)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3)
        k = cv2.waitKey(20) & 0xFF
        if coordinateStore1.clicks == 0:
            write_boxes(current_image, width, "Top left", "Blue")
        elif coordinateStore1.clicks == 1:
            write_boxes(current_image, width, "Top left", "Red")
        elif coordinateStore1.clicks == 2:
            write_boxes(current_image, width, "Top right", "Blue")
        elif coordinateStore1.clicks == 3:
            write_boxes(current_image, width, "Top right", "Red")
        elif coordinateStore1.clicks == 4:
            write_boxes(current_image, width, "Bottom left", "Blue")
        elif coordinateStore1.clicks == 5:
            write_boxes(current_image, width, "Bottom left", "Red")
        elif coordinateStore1.clicks == 6:
            write_boxes(current_image, width, "Bottom right", "Blue")
        elif coordinateStore1.clicks == 7:
            write_boxes(current_image, width, "Bottom right", "Red")
        
        if k == 109:
            coordinateStore1.skip_point()
        
        if k == 32:
            count = 1
            for i in coordinateStore1.points:
                if (count % 2) == 0:
                    temp_df=pd.DataFrame({"Name": [image_name], "Position": [pos_name], "Point": ["red"], "Coordinates": [i]})
                else:
                    temp_df=pd.DataFrame({"Name": [image_name], "Position": [pos_name], "Point": ["blue"], "Coordinates": [i]})
                df = pd.concat([df,temp_df], ignore_index = True)
                if count == 8:
                    break
                count += 1
            coordinateStore1.__init__()
            break
        
        if k == 110:
            break
        
        if k == 27:
            Stop = True
            break
    
    if Stop == True:
        break
            
cv2.destroyWindow('image')
cv2.destroyAllWindows()

if os.path.exists("OF_mirror/Lid_coords.csv"):
    df.to_csv("OF_mirror/Lid_coords.csv", index=False, header=False, mode="a")
else:
    df.to_csv("OF_mirror/Lid_coords.csv", index=False)

print("done")